In [5]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('altoromutual.csv', dtype=str)

df.head()

df2 = pd.read_csv('nikto_testfire_scan_1759569379.csv')
df2

Nikto - v2.1.5/2.1.5
testfire.net 65.61.137.117                          80         NaN     NaN                  NaN                                                   Apache-Coyote/1.1
             65.61.137.117SCALAR(0x6536b91a5f88)80" OSVDB-0    GET     /                    The anti-clickjacking X-Frame-Options header is...                  NaN
                                                                                            Cookie JSESSIONID created without the httponly ...                  NaN
                                                               OPTIONS /                    Allowed HTTP Methods: GET, HEAD, POST, PUT, DEL...                  NaN
                                                    OSVDB-397  GET     /                    HTTP method ('Allow' Header): 'PUT' method coul...                  NaN
                                                    OSVDB-5646 GET     /                    HTTP method ('Allow' Header): 'DELETE' may allo...                  NaN
                                                    OSVDB-0    DEBUG   HASH(0x6536b92ac6b8) DEBUG HTTP verb may show server debugging infor...                  NaN
                                                               GET     /lpt9                Server leaks inodes via ETags, header found wit...                  NaN

In [ ]:
out_rows = []

for index, r in df2.iterrows():
    # df2 expected columns: host, ip, port, method, path, description
    host, ip, port, _, _, description = index 
    out = {
        "plugin_id": None,
        "cve_id": None,
        "CVSS_v2.0_base_score": None,
        "risk": None,  # You can set severity based on keywords later
        "host": [host, ip],
        "protocol": "HTTP",
        "port": port,
        "name": None,
        "synopsis": None,
        "description": description, 
        "solution": None,
        "cvss_v3": None,
        "cvss_v2": None,
        "vpr": None,
        "epss": None,
        "age_days": None,
        "core_impact": None,
        "source": "nikto",
        # "real_row": r.to_dict()
    }
    out_rows.append(out)

out_df = pd.DataFrame(out_rows)
out_df.to_csv('nikto_converted2.csv', index=True)

out_rows[:2]  # Show first 2 rows of the converted data

In [11]:
out_rows = []

for index, r in df.iterrows():
    cvss_v3_val = r.get('CVSS v3.0 Base Score')
    cvss_v3 = np.float32(cvss_v3_val) if cvss_v3_val not in [None, 'nan', ''] else None

    cvss_v2_val = r.get('CVSS v2.0 Base Score')
    cvss_v2 = np.float32(cvss_v2_val) if cvss_v2_val not in [None, 'nan', ''] else None

    vpr_val = r.get('VPR Score')
    vpr = np.float32(vpr_val) if vpr_val not in [None, 'nan', ''] else None

    epss_val = r.get('EPSS Score')
    epss = np.float32(epss_val) if epss_val not in [None, 'nan', ''] else None

    pub_date_str = r.get('Plugin Publication Date')
    mod_date_str = r.get('Plugin Modification Date')

    # Parse dates
    pub_date = datetime.strptime(pub_date_str, "%Y/%m/%d")
    mod_date = datetime.strptime(mod_date_str, "%Y/%m/%d")

    # Today's date
    today = datetime.utcnow()

    # Calculate ages
    age_since_publish = (today - pub_date).days
    age_since_mod = (today - mod_date).days
    out = {
        "plugin_id": r['Plugin ID'],
        "cve_id": r['CVE'] if r['CVE'] != 'nan' else None,
        'CVSS_v2.0_base_score': r['CVSS v2.0 Base Score'] if r['CVSS v2.0 Base Score'] != 'nan' else None,
        'risk': r['Risk'] if r['Risk'] != 'nan' else None,
        'host': r['Host'],
        'protocol' : r['Protocol'],
        'port': r['Port'] if r['Port'].isdigit() else None,
        'name': r['Name'],
        'synopsis': r['Synopsis'],
        'description': r['Description'] if r['Description'] != 'nan' else None,
        'solution': r['Solution'] if r['Solution'] != 'nan' else None,
        "cvss_v3": cvss_v3,
        "cvss_v2": cvss_v2,
        "vpr": vpr,
        "epss": epss,
        'age_days' : age_since_publish,
        'core_impact' : r['Core Impact'] if r['Core Impact'] != 'nan' else None,
        'real_row' : r.to_dict()
    }
    out_rows.append(out)

for index, r in df2.iterrows():
# df2 expected columns: host, ip, port, method, path, description
    host, ip, port, _, _, description = index 
    out = {
        "plugin_id": None,
        "cve_id": None,
        "CVSS_v2.0_base_score": None,
        "risk": None,  # You can set severity based on keywords later
        "host": [host, ip],
        "protocol": "HTTP",
        "port": port,
        "name": None,
        "synopsis": None,
        "description": description, 
        "solution": None,
        "cvss_v3": None,
        "cvss_v2": None,
        "vpr": None,
        "epss": None,
        "age_days": None,
        "core_impact": None,
        "source": "nikto",
        # "real_row": r.to_dict()
    }
    out_rows.append(out)

out_df = pd.DataFrame(out_rows)
out_df.to_csv("normalized_findings.csv", index=True)

C:\Users\SK Mahesh\AppData\Local\Temp\ipykernel_17792\2467106368.py:24: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow()


In [17]:
out_df.head()

,plugin_id,cve_id,CVSS_v2.0_base_score,risk,host,protocol,port,name,synopsis,description,solution,cvss_v3,cvss_v2,vpr,epss,age_days,core_impact,real_row
0,10107,NaN,NaN,NaN,65.61.137.117,tcp,80,HTTP Server Type and Version,A web server is running on the remote host.,This plugin attempts to determine the type and...,NaN,NaN,NaN,NaN,NaN,9402,NaN,"{'Plugin ID': '10107', 'CVE': nan, 'CVSS v2.0 ..."
1,10107,NaN,NaN,NaN,65.61.137.117,tcp,443,HTTP Server Type and Version,A web server is running on the remote host.,This plugin attempts to determine the type and...,NaN,NaN,NaN,NaN,NaN,9402,NaN,"{'Plugin ID': '10107', 'CVE': nan, 'CVSS v2.0 ..."
2,10107,NaN,NaN,NaN,65.61.137.117,tcp,8080,HTTP Server Type and Version,A web server is running on the remote host.,This plugin attempts to determine the type and...,NaN,NaN,NaN,NaN,NaN,9402,NaN,"{'Plugin ID': '10107', 'CVE': nan, 'CVSS v2.0 ..."
3,10114,CVE-1999-0524,2.1,Low,65.61.137.117,icmp,0,ICMP Timestamp Request Remote Date Disclosure,It is possible to determine the exact time set...,The remote host answers to an ICMP timestamp r...,"Filter out the ICMP timestamp requests (13), a...",NaN,2.1,2.2,0.0037,9558,NaN,"{'Plugin ID': '10114', 'CVE': 'CVE-1999-0524',..."
4,10287,NaN,NaN,NaN,65.61.137.117,udp,0,Traceroute Information,It was possible to obtain traceroute information.,Makes a traceroute to the remote host.,NaN,NaN,NaN,NaN,NaN,9440,NaN,"{'Plugin ID': '10287', 'CVE': nan, 'CVSS v2.0 ..."


In [19]:
import networkx as nx
import math
import json

# --- Example sample data (replace with your normalized outputs) ---
nodes = [
    # asset_id, compromise probability (if unknown, use 0.01)
    ("web-01", {"compromise_prob": 0.6, "role":"web", "internet_exposed": True}),
    ("app-02", {"compromise_prob": 0.3, "role":"app", "internet_exposed": False}),
    ("db-01", {"compromise_prob": 0.1, "role":"db", "internet_exposed": False}),
    ("internal-01", {"compromise_prob": 0.05, "role":"workstation", "internet_exposed": False}),
]

edges = [
    # from, to, traverse_prob, edge_type
    ("web-01", "app-02", {"traverse_prob": 0.8, "edge_type":"http->app"}),
    ("app-02", "db-01", {"traverse_prob": 0.9, "edge_type":"app->db"}),
    ("internal-01", "app-02", {"traverse_prob": 0.5, "edge_type":"ssh"}),
    ("web-01", "internal-01", {"traverse_prob": 0.2, "edge_type":"pivot"}),
]

# --- Build graph ---
G = nx.DiGraph()
for nid, attrs in nodes:
    # ensure prob bounds and fallback
    p_node = float(attrs.get("compromise_prob", 0.01))
    p_node = min(max(p_node, 1e-6), 0.999999)
    G.add_node(nid, compromise_prob=p_node, **{k:v for k,v in attrs.items() if k != "compromise_prob"})

for a, b, attrs in edges:
    p_edge = float(attrs.get("traverse_prob", 0.1))
    p_edge = min(max(p_edge, 1e-6), 0.999999)
    G.add_edge(a, b, traverse_prob=p_edge, **{k:v for k,v in attrs.items() if k != "traverse_prob"})

# --- Path scoring function ---
def path_probability(G, path):
    """
    Compute the probability of successfully traversing the entire path.
    For first node (entry): use its compromise probability
    For each edge + destination: multiply edge traverse_prob * destination compromise_prob
    """
    if len(path) == 0:
        return 0.0
    
    # Start with entry node compromise probability
    prob = G.nodes[path[0]].get("compromise_prob", 1e-6)
    
    # For each edge in the path
    for i in range(len(path) - 1):
        src = path[i]
        dst = path[i + 1]
        edge_p = G[src][dst].get("traverse_prob", 1e-6)
        dst_node_p = G.nodes[dst].get("compromise_prob", 1e-6)
        # Probability to traverse edge AND compromise destination
        prob *= (edge_p * dst_node_p)
    
    return prob

# Compute combined cost for each edge (for Dijkstra's algorithm)
for u, v, data in G.edges(data=True):
    dst_p = G.nodes[v]["compromise_prob"]
    edge_p = data["traverse_prob"]
    # Cost = -log(edge_traverse_prob * dest_compromise_prob)
    # Add small epsilon to avoid log(0)
    cost = -math.log(edge_p * dst_p + 1e-12)
    G[u][v]['combined_cost'] = cost

# Use Dijkstra to find best paths
def best_path(G, source_candidates, target, k=3):
    """
    Find the top k highest-probability paths from any source candidate to target.
    """
    paths = []
    
    for src in source_candidates:
        if src not in G:
            continue
        if target not in G:
            continue
            
        try:
            # Find shortest path by combined_cost (which represents highest probability)
            path = nx.shortest_path(G, source=src, target=target, weight='combined_cost')
            # Compute actual path probability
            prob = path_probability(G, path)
            # Compute total cost for reference
            total_cost = sum(G[path[i]][path[i+1]]['combined_cost'] for i in range(len(path) - 1))
            
            paths.append({
                "source": src,
                "target": target,
                "path": path,
                "probability": prob,
                "cost": total_cost
            })
        except nx.NetworkXNoPath:
            # No path exists from this source to target
            continue
    
    # Sort by probability descending (highest probability first)
    paths = sorted(paths, key=lambda x: x['probability'], reverse=True)
    return paths[:k]

# Example: find best paths from internet-exposed nodes (entry points)
entry_nodes = [n for n, d in G.nodes(data=True) if d.get("internet_exposed")]
target = "db-01"

print(f"Entry nodes (internet-exposed): {entry_nodes}")
print(f"Target: {target}\n")

best = best_path(G, entry_nodes, target, k=5)

# Pretty-print results
print("Top Attack Paths:\n")
print(json.dumps(best, indent=2))


Entry nodes (internet-exposed): ['web-01']
Target: db-01

Top Attack Paths:

[
  {
    "source": "web-01",
    "target": "db-01",
    "path": [
      "web-01",
      "app-02",
      "db-01"
    ],
    "probability": 0.012960000000000001,
    "cost": 3.83506196427674
  }
]
